# 01 — Trust: Data Quality & Reliability

This notebook demonstrates **AI Readiness Pillar #1: Trust**.

You will:
- Load intentionally imperfect raw data
- Define a **data contract** (schema + rules)
- Use **Great Expectations** to validate data
- Apply a **quality gate** (fail or quarantine)
- Produce a cleaned "Silver" dataset

> Works in **VS Code** (local) and **Azure Databricks**.


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# Set this for local VS Code (default repo layout)
BASE_PATH = Path("..")  # if running from notebooks/
DATA_RAW = BASE_PATH / "data" / "raw"
DATA_CURATED = BASE_PATH / "data" / "curated"

DATA_RAW, DATA_CURATED


In [ ]:
customers = pd.read_csv(DATA_RAW / "customers.csv")
orders = pd.read_csv(DATA_RAW / "orders.csv")
events = pd.read_csv(DATA_RAW / "events.csv")

customers.head(), orders.head(), events.head()


## Define a simple data contract

A data contract is a **shared agreement** between producers and consumers:
- expected columns + types
- required fields
- allowed values / ranges
- uniqueness constraints

Below we define a lightweight contract in Python (you can store this as YAML/JSON in real systems).

In [ ]:
customer_contract = {
    "required_columns": ["customer_id","email","department","country","created_at"],
    "unique": ["customer_id"],
    "not_null": ["customer_id","department","created_at"],
}

order_contract = {
    "required_columns": ["order_id","customer_id","amount","currency","order_ts"],
    "unique": ["order_id"],
    "not_null": ["order_id","customer_id","amount","order_ts"],
    "ranges": {"amount": (0, None)},  # no negative amounts
}

contracts = {"customers": customer_contract, "orders": order_contract}
contracts


## Validate with Great Expectations

Great Expectations provides reusable expectation suites.

Install:
- `pip install great_expectations`


In [ ]:
import great_expectations as ge

ge_customers = ge.from_pandas(customers)
ge_orders = ge.from_pandas(orders)

# Expectations (examples)
results = []
results.append(ge_customers.expect_table_columns_to_match_ordered_list(customer_contract["required_columns"]))
results.append(ge_customers.expect_column_values_to_be_unique("customer_id"))
results.append(ge_customers.expect_column_values_to_not_be_null("customer_id"))
results.append(ge_customers.expect_column_values_to_not_be_null("created_at"))

results.append(ge_orders.expect_column_values_to_be_unique("order_id"))
results.append(ge_orders.expect_column_values_to_not_be_null("amount"))
results.append(ge_orders.expect_column_values_to_be_between("amount", min_value=0))

# Summarise
pd.DataFrame([{"expectation": r["expectation_config"]["expectation_type"], "success": r["success"]} for r in results])


## Quality Gate: quarantine bad rows

Instead of "hoping" downstream systems handle issues, implement **quality gates**.

Here we:
- fix a few simple issues
- quarantine records that still violate hard rules
- write Silver outputs


In [ ]:
# Basic cleaning examples
customers_clean = customers.copy()

# standardise country codes a bit
customers_clean["country"] = customers_clean["country"].replace({
    "United Kingdom": "UK",
    "U.K.": "UK",
    "": np.nan
})

# drop duplicate customer_id keeping first
customers_clean = customers_clean.drop_duplicates(subset=["customer_id"], keep="first")

# quarantine: missing critical fields
cust_quarantine = customers_clean[customers_clean["department"].isna() | customers_clean["created_at"].isna()].copy()
customers_silver = customers_clean.drop(cust_quarantine.index).copy()

orders_clean = orders.copy()
order_quarantine = orders_clean[(orders_clean["amount"] < 0) | (orders_clean["currency"].isna())].copy()
orders_silver = orders_clean.drop(order_quarantine.index).copy()

len(customers), len(customers_silver), len(cust_quarantine), len(orders), len(orders_silver), len(order_quarantine)


In [ ]:
DATA_CURATED.mkdir(parents=True, exist_ok=True)

customers_silver.to_csv(DATA_CURATED / "customers_silver.csv", index=False)
orders_silver.to_csv(DATA_CURATED / "orders_silver.csv", index=False)

cust_quarantine.to_csv(DATA_CURATED / "customers_quarantine.csv", index=False)
order_quarantine.to_csv(DATA_CURATED / "orders_quarantine.csv", index=False)

print("Wrote curated datasets to:", DATA_CURATED.resolve())


## Why this matters for AI

- **RAG**: noisy docs/metadata → wrong retrieval → hallucinations
- **Agents**: bad data → wrong actions at scale
- **Analytics-to-action**: inconsistent definitions → conflicting decisions

Quality gates make AI systems **predictable**.
